In [1]:
epub_path = '../../Resources/مجنون ليلى.epub'
book_title = 'مجنون ليلى'
example_xhtml = "chapter-1-2-L.xhtml"

## This function is general for any EPUB file

In [34]:
import zipfile
from bs4 import BeautifulSoup
import os

def extract_and_save_chapters(epub_path, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    with zipfile.ZipFile(epub_path, 'r') as zip_ref:
        zip_ref.extractall("temp_epub_extraction")

    chapter_counter = 1
    for root, dirs, files in sorted(os.walk("temp_epub_extraction")):
        for file in sorted(files):
            if file.endswith('.html') or file.endswith('.xhtml'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    soup = BeautifulSoup(f, 'lxml')
                    texts = soup.get_text(separator='\n', strip=True)
                    chapter_filename = f"chapter_{chapter_counter}.txt"
                    chapter_path = os.path.join(output_directory, chapter_filename)
                    with open(chapter_path, 'w', encoding='utf-8') as chapter_file:
                        chapter_file.write(texts)
                    chapter_counter += 1
    
    # Cleanup extraction directory
    for root, dirs, files in os.walk("temp_epub_extraction", topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))
    os.rmdir("temp_epub_extraction")

# Specify the directory where you want to save the chapters
output_directory = f'./{book_title}_chapters'

extract_and_save_chapters(epub_path, output_directory)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14528\1872953677.py:18: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(f, 'lxml')


In [19]:
from lxml import etree

# Load the XHTML content from the file
with open(example_xhtml, "rb") as file:
    # Parse the XHTML
    tree = etree.parse(file)


# Use list comprehension to get all elements
    ## it checks if the element has text and if the text is not empty (whitespace)
elements_list = [element for element in tree.iter() if element.text and
                element.text.strip() and
                element.text.strip() != ':' and
                element.tag != '{http://www.w3.org/1999/xhtml}a']

characters_names = list(set([element.text for element in elements_list if element.tag == '{http://www.w3.org/1999/xhtml}b']))

In [21]:
speakers = []
speakers.extend(characters_names)
speakers.append('المعلق')

current_speaker = 'المعلق'
combined_dialogue = ""
last_speaker = None

# A list of tuples that will store the speaker and the dialogue
dialogues_and_speakers = []

for element in elements_list:
    cleaned_text = element.text.strip()  # Clean up the text for processing

    # Check if the element's tag indicates a speaker
    if element.tag == '{http://www.w3.org/1999/xhtml}b':
        if cleaned_text in speakers:
            # Announce the speaker's name by the narrator before the speaker's dialogue
            dialogues_and_speakers.append((cleaned_text, "المعلق"))
            current_speaker = cleaned_text  # Update the current speaker
        else:
            current_speaker = 'المعلق'  # Reset to narrator if the speaker is not recognized
    elif cleaned_text.startswith("(") and cleaned_text.endswith(")"):
        # Special handling for narrative text
        dialogues_and_speakers.append((cleaned_text, "المعلق"))
        # Reset the current speaker to the narrator
        dialogues_and_speakers.append(("", current_speaker))
    else:
        # Normal dialogue
        if current_speaker == last_speaker:
            # If the current speaker is the same as the last, combine the dialogue lines
            last_dialogue, last_speaker_in_list = dialogues_and_speakers[-1]
            combined_dialogue = last_dialogue + " " + cleaned_text
            dialogues_and_speakers[-1] = (combined_dialogue, last_speaker_in_list)
        else:
            # If the speaker changes, just append the dialogue
            dialogues_and_speakers.append((cleaned_text, current_speaker))
        last_speaker = current_speaker  # Update the last_speaker to the current one

# remove any empty dialogues
dialogues_and_speakers = [(dialogue, speaker) for dialogue, speaker in dialogues_and_speakers if dialogue.strip()]

# Printing dialogues and their speakers
for dialogue, speaker in dialogues_and_speakers:
    print(f"{speaker}: {dialogue}\n")

# save this list to a file as a npy
import numpy as np

np.save('dialogues_and_speakers.npy', dialogues_and_speakers)

المعلق: مجنون ليلى الفصل الأول

المعلق: (ساحة أمام خيام المهدي في حي بني عامر - مجلس من مجالس السمر في هذه
            الساحة - فتية وفتيات من الحي يسمرون في أوائل الليل، وفي أيدي الفتيات صوف ومغازل يلهون
            بها وهم يتحدثون — تخرج ليلى من خيام أبيها عند ارتفاع الستار ويدها في يد ابن ذريح)

المعلق: ليلى

ليلى: دعي الغزْلَ سلمى وحَيِّي معي منارَ الحِجَازِ فَتَى يَثْرِبِ

المعلق: (تصافحه سلمى)

ليلى:  ويا هِنْدُ هذا أديبُ الحِجازِ هلمِّي بمَقْدَمِهِ رَحِّبِي

المعلق: (تصافحه هند ويحتفي به السامرون)

المعلق: سعد

سعد: أمن يثربٍ أنت آتٍ؟

المعلق: ابن ذريح

ابن ذريح: أجل من البلدِ القُدُس الطيِّب

المعلق: ليلى

ليلى: أيابنَ ذَريحٍ لقينا الغمام

المعلق: هند

هند: وطَافتْ بنا نَفَحَاتُ النبي

المعلق: عبلة

المعلق: (هامسة إلى سعد)

عبلة: مَن ابْنُ ذَريحٍ؟

المعلق: سعد

سعد: فتًى ذِكرُه على مَشرِق الشمس والمغرب رَضيعُ الحُسَيْنِ عليه السلامُ وترْبُ الحُسَيْنِ من المكتبِ

المعلق: عبلة

المعلق: (إلى بشر ومشير إلى ابن ذريح)

عبلة: أتسمَعُ بشرُ رضيعُ الْحُسَيْنِ فديْتُ الرضيعيْن والمُرضعهْ 

In [33]:
import re

dialogues = [dialogue for dialogue, speaker in dialogues_and_speakers]

narrative_texts = []
for i in range(len(dialogues)):
    if dialogues[i].startswith('(') and dialogues[i].endswith(')'):

        # Check if the previous element is a character name and append accordingly
        if i > 0 and dialogues[i-1].lower() in characters_names:
            narrative_texts.append(f"{dialogues[i-1]} {dialogues[i]}")
        else:
            narrative_texts.append(dialogues[i])

# delete all \n from the narrative text
cleaned_narrative_texts = [narrative.replace('\n', ' ') for narrative in narrative_texts]

# print length of the narrative text
print(len(narrative_texts))

cleaned_narrative_texts = [re.sub(r'\s{2,}', ' ', narrative) for narrative in cleaned_narrative_texts]

# print the narrative text
for narrative in narrative_texts:
    print(narrative)

# print the cleaned narrative text
for narrative in cleaned_narrative_texts:
    print(narrative)

# save the cleaned narrative text to a file
with open('narrative_texts.txt', 'w', encoding='utf-8') as file:
    for narrative in cleaned_narrative_texts:
        file.write(narrative + '\n')



58
(ساحة أمام خيام المهدي في حي بني عامر - مجلس من مجالس السمر في هذه
            الساحة - فتية وفتيات من الحي يسمرون في أوائل الليل، وفي أيدي الفتيات صوف ومغازل يلهون
            بها وهم يتحدثون — تخرج ليلى من خيام أبيها عند ارتفاع الستار ويدها في يد ابن ذريح)
(تصافحه سلمى)
(تصافحه هند ويحتفي به السامرون)
عبلة (هامسة إلى سعد)
عبلة (إلى بشر ومشير إلى ابن ذريح)
بشر (هامسًا ومتلفتًا كأنما يخشى أن يسمعه أحد)
(تتضاحك الفتيات وتقول إحداهن لأخرى)
هند (ساخرة)
(تحاول ليلى أن تمد رجلها فتتألم وتستغيث)
هند (متهكمة)
(لنفسها)
عبلة (ضجرة)
منازل (ضاحكًا)
(يبدو على ليلى شيء من الزهو فتتهامس الفتيات)
هند (مشيرة إلى ليلى)
(فإنك لي جارٌ ولا ترهبِ الدَّهْرَا)
ليلى (ضاحكة)
(ضحك من الجميع)
(ثم تقول في لوعة وصوت مخفوض وكأنما تحدث نفسها)
(تسترسل في حديثها الأول)
بشر (مندفعًا بحماسة!)
أصوات (بين الضحك والسخرية)
بشر (ساخرًا)
ليلى (غاضبة)
(تهيب بالسامرين وقد بلغ بها الغضب أقصاه)
ابن ذريح (متوسلًا)
(تدخل خباءها بينما ينفض السامرون فلا يتثاقل منهم في القيام إلا
                منازل — الهرج والأسف يسودان الجميع)


In [35]:
approved_narrative_texts = []

for index, text in enumerate(cleaned_narrative_texts):
    user_input = input(f"Do you like: \n{text}\nEnter 'y' for yes, anything else for no: ")
    if user_input.lower() == 'y':
        approved_narrative_texts.append((index, text))

for narrative in approved_narrative_texts:
    print(narrative)

(0, '(ساحة أمام خيام المهدي في حي بني عامر - مجلس من مجالس السمر في هذه الساحة - فتية وفتيات من الحي يسمرون في أوائل الليل، وفي أيدي الفتيات صوف ومغازل يلهون بها وهم يتحدثون — تخرج ليلى من خيام أبيها عند ارتفاع الستار ويدها في يد ابن ذريح)')
(3, 'عبلة (هامسة إلى سعد)')
(5, 'بشر (هامسًا ومتلفتًا كأنما يخشى أن يسمعه أحد)')
(6, '(تتضاحك الفتيات وتقول إحداهن لأخرى)')
(7, 'هند (ساخرة)')
(8, '(تحاول ليلى أن تمد رجلها فتتألم وتستغيث)')
(9, 'هند (متهكمة)')
(11, 'عبلة (ضجرة)')
(12, 'منازل (ضاحكًا)')
(13, '(يبدو على ليلى شيء من الزهو فتتهامس الفتيات)')
(14, 'هند (مشيرة إلى ليلى)')
(16, 'ليلى (ضاحكة)')
(17, '(ضحك من الجميع)')
(20, 'بشر (مندفعًا بحماسة!)')
(21, 'أصوات (بين الضحك والسخرية)')
(22, 'بشر (ساخرًا)')
(23, 'ليلى (غاضبة)')
(25, 'ابن ذريح (متوسلًا)')
(27, 'سعد (ضاحكًا)')
(28, '(تسمع ضحكاتهم من أقصى الطريق بينما يظهر قيس وزياد من جانب المسرح الآخر)')
(29, 'منازل (وقد سمع همهمة الصوت ورأى شبحيهما في الظلام)')
(31, 'قيس (حنقًا)')
(32, 'منازل (ساخرًا)')
(33, 'زياد (مغضبًا)')
(34, 'منازل (وقد أ

In [36]:
# save the approved narrative texts to a file without the index
with open('approved_narrative_texts.txt', 'w', encoding='utf-8') as file:
    for index, narrative in approved_narrative_texts:
        file.write(f"{narrative}\n")

In [42]:
unmodified_narrative_texts = [dialogue for dialogue in dialogues if dialogue.startswith('(') and dialogue.endswith(')')]
uncleaned_narrative_texts = [unmodified_narrative_texts[index] for index, narrative in approved_narrative_texts]

# save the uncleaned narrative texts to a npy file
np.save('uncleaned_narrative_texts.npy', uncleaned_narrative_texts)

In [43]:
# use uncleaned_narrative_texts.npy to loop through the narrative texts and print them
narrative_texts = np.load('uncleaned_narrative_texts.npy', allow_pickle=True)

# enumerate the narrative texts
for index, narrative in enumerate(narrative_texts):
    print(f"{index+1}: {narrative}")


1: (ساحة أمام خيام المهدي في حي بني عامر - مجلس من مجالس السمر في هذه
            الساحة - فتية وفتيات من الحي يسمرون في أوائل الليل، وفي أيدي الفتيات صوف ومغازل يلهون
            بها وهم يتحدثون — تخرج ليلى من خيام أبيها عند ارتفاع الستار ويدها في يد ابن ذريح)
2: (هامسة إلى سعد)
3: (هامسًا ومتلفتًا كأنما يخشى أن يسمعه أحد)
4: (تتضاحك الفتيات وتقول إحداهن لأخرى)
5: (ساخرة)
6: (تحاول ليلى أن تمد رجلها فتتألم وتستغيث)
7: (متهكمة)
8: (ضجرة)
9: (ضاحكًا)
10: (يبدو على ليلى شيء من الزهو فتتهامس الفتيات)
11: (مشيرة إلى ليلى)
12: (ضاحكة)
13: (ضحك من الجميع)
14: (مندفعًا بحماسة!)
15: (بين الضحك والسخرية)
16: (ساخرًا)
17: (غاضبة)
18: (متوسلًا)
19: (ضاحكًا)
20: (تسمع ضحكاتهم من أقصى الطريق بينما يظهر قيس وزياد من جانب المسرح
                الآخر)
21: (وقد سمع همهمة الصوت ورأى شبحيهما في الظلام)
22: (حنقًا)
23: (ساخرًا)
24: (مغضبًا)
25: (وقد أخذ بتلابيبه)
26: (خارجًا من الخباء)
27: (خجلًا)
28: (دهشًا)
29: (مناديًا)
30: (من أقصى الخباء)
31: (تظهر ليلى على باب الخباء)
32: (تخرج عفراء وتتبعها ليلى)


In [40]:
character_descriptions1 = {
    "قيس": "Eye Color: dark brown, Eye Shape: deep-set, Face Shape: oval, Nose: straight, Hairstyle: long and loose, Hair Color: black, Skin Color: light brown, Lips: thin, Eyebrows: thick and arched, Jawline: pronounced, Age: mid-20s, Gender: Male, Race: Human, Clothing: White ankle-length robe with straight collar and simple embroidery.",
    "ليلى": "Eye Color: dark brown, Eye Shape: almond, Face Shape: heart, Nose: small, upturned, Hairstyle: long and wavy, Hair Color: black, Skin Color: fair, Lips: medium, Eyebrows: medium, curved, Jawline: soft, Age: early 20s, Gender: Female, Race: Human, Clothing: Ankle-length soft blue dress with minimal floral embroidery around the neckline and sleeves.",
    "المهدي": "Eye Color: dark gray, Eye Shape: round, Face Shape: square, Nose: broad, Hairstyle: balding, Hair Color: gray, Skin Color: olive, Lips: straight, Eyebrows: full and bushy, Jawline: strong, Age: late 50s, Gender: Male, Race: Human, Clothing: Dark green robe with a thick woolen cloak, decorated with simple geometric patterns along the edges.",
    "ورد": "Eye Color: green, Eye Shape: round, Face Shape: rectangular, Nose: hooked, Hairstyle: short, Hair Color: dark brown, Skin Color: tan, Lips: narrow, Eyebrows: straight, Jawline: square, Age: mid-30s, Gender: Male, Race: Human, Clothing: Sturdy brown leather jerkin over a thick green cotton tunic, suitable for travel and outdoor activities.",
    "ابن عوف": "Eye Color: hazel, Eye Shape: round, Face Shape: round, Nose: prominent, Hairstyle: short, Hair Color: salt and pepper, Skin Color: medium, Lips: full, Eyebrows: thick, Jawline: full, Age: late 40s, Gender: Male, Race: Human, Clothing: Formal military uniform in dark blue with red trim and gold insignia, complete with a ceremonial sword and boots."
}

character_descriptions2 = {
    "زياد": "Eye Color: light brown, Eye Shape: oval, Face Shape: oval, Nose: average, Hairstyle: medium length, Hair Color: black, Skin Color: medium, Lips: medium thickness, Eyebrows: moderately thick, Jawline: gently rounded, Age: late 20s, Gender: Male, Race: Human, Clothing: Simple white cotton shirt under a light brown vest, dark brown trousers, practical for a scribe.",
    "منازل": "Eye Color: dark brown, Eye Shape: narrow, Face Shape: long, Nose: long, Hairstyle: kept short, Hair Color: dark brown, Skin Color: olive, Lips: thin, Eyebrows: thin and straight, Jawline: sharp, Age: mid-20s, Gender: Male, Race: Human, Clothing: Dark green robe with minimal decoration, emphasizing a somber and serious demeanor.",
    "بشر": "Eye Color: black, Eye Shape: wide, Face Shape: square, Nose: wide, Hairstyle: bald, Hair Color: none, Skin Color: dark, Lips: wide, Eyebrows: bushy, Jawline: strong, Age: early 40s, Gender: Male, Race: Human, Clothing: Heavy armor suited for battle, dark metal with simple engravings, complemented by a red woolen cape.",
    "ابن ذريح": "Eye Color: amber, Eye Shape: slightly slanted, Face Shape: heart, Nose: refined, Hairstyle: long and flowing, Hair Color: golden brown, Skin Color: fair, Lips: well-defined, Eyebrows: arched, Jawline: defined, Age: mid-30s, Gender: Male, Race: Human, Clothing: Elegant blue robe with intricate silver embroidery, typical of a respected poet.",
    "نصيب": "Eye Color: grey, Eye Shape: round, Face Shape: round, Nose: small, Hairstyle: short, Hair Color: light brown, Skin Color: light, Lips: small, Eyebrows: sparse, Jawline: soft, Age: early 30s, Gender: Male, Race: Human, Clothing: Neatly tailored beige tunic with brown leather belt, appropriate for a scribe or clerk."
}

character_descriptions3 = {
    "سعد": "Eye Color: black, Eye Shape: round, Face Shape: round, Nose: broad, Hairstyle: shaved head, Hair Color: none, Skin Color: deep brown, Lips: full, Eyebrows: dense, Jawline: broad, Age: mid-30s, Gender: Male, Race: Human, Clothing: Sturdy brown tunic and dark trousers, practical for desert environments, accessorized with a leather belt and boots.",
    "الغريض": "Eye Color: hazel, Eye Shape: almond, Face Shape: triangular, Nose: pointed, Hairstyle: shoulder-length, Hair Color: chestnut, Skin Color: light olive, Lips: narrow, Eyebrows: lightly arched, Jawline: chiseled, Age: late 20s, Gender: Male, Race: Human, Clothing: Flamboyant purple robe with gold trim, suited to a famed performer, complete with matching hat and decorative scarf.",
    "ابن سعيد": "Eye Color: deep blue, Eye Shape: narrow, Face Shape: oval, Nose: straight, Hairstyle: short and curly, Hair Color: dark brown, Skin Color: fair, Lips: medium, Eyebrows: thick and curved, Jawline: well-defined, Age: early 40s, Gender: Male, Race: Human, Clothing: Formal poet's attire, dark green cloak over a cream-colored tunic, adorned with poetic scrolls and a feathered cap.",
    "أمية": "Eye Color: green, Eye Shape: large, Face Shape: diamond, Nose: aquiline, Hairstyle: medium, wavy, Hair Color: black, Skin Color: tan, Lips: thin, Eyebrows: defined, Jawline: angular, Age: mid-30s, Gender: Male, Race: Human, Clothing: Simple merchant's garb, dark brown vest over a white shirt, dark trousers, suitable for travel and trade.",
    "الأموي": "Eye Color: red, Eye Shape: piercing, Face Shape: elongated, Nose: narrow, Hairstyle: bald, Hair Color: none, Skin Color: pale, Lips: thin, Eyebrows: non-existent, Jawline: pointed, Age: ageless, Gender: Male, Race: Demon, Clothing: Dark robe with red accents, flowing and ethereal, giving an ominous presence."
}

character_descriptions4= {
    "عضرفوت": "Eye Color: yellow, Eye Shape: slitted, Face Shape: narrow, Nose: flat, Hairstyle: none, Hair Color: none, Skin Color: gray, Lips: thin, Eyebrows: low and straight, Jawline: sharp, Age: indeterminate, Gender: Male, Race: Demon, Clothing: Tattered dark cloak with frayed edges, exuding a sinister aura.",
    "هبيد": "Eye Color: black, Eye Shape: deep-set, Face Shape: square, Nose: wide, Hairstyle: none, Hair Color: none, Skin Color: dark gray, Lips: wide, Eyebrows: bushy, Jawline: strong, Age: ageless, Gender: Male, Race: Demon, Clothing: Heavy, rugged armor with dark metal plates, designed for intimidation and protection.",
    "عسر": "Eye Color: red, Eye Shape: narrow, Face Shape: gaunt, Nose: sharp, Hairstyle: none, Hair Color: none, Skin Color: pale, Lips: cracked, Eyebrows: thin and arched, Jawline: emaciated, Age: ageless, Gender: Male, Race: Demon, Clothing: Long, flowing black robe, with a hood that casts the face in shadow, enhancing its ghastly appearance.",
    "عاصف": "Eye Color: bright green, Eye Shape: wide, Face Shape: round, Nose: small, Hairstyle: spiky, Hair Color: fiery red, Skin Color: light, Lips: small, Eyebrows: thin, Jawline: soft, Age: unknown, Gender: Male, Race: Demon, Clothing: Vibrant red tunic with swirling orange patterns, matching the dynamic and tempestuous nature of the character.",
    "بلهاء": "Eye Color: brown, Eye Shape: large, Face Shape: oval, Nose: delicate, Hairstyle: long and straight, Hair Color: dark brown, Skin Color: medium, Lips: full, Eyebrows: neatly shaped, Jawline: gentle, Age: mid-20s, Gender: Female, Race: Human, Clothing: Soft green gown with delicate lace detailing at the cuffs and hem, simple yet elegant."
}

character_descriptions5 = {
    "عفراء": "Eye Color: hazel, Eye Shape: oval, Face Shape: heart, Nose: small, Hairstyle: long and wavy, Hair Color: light brown, Skin Color: fair, Lips: medium, Eyebrows: arched, Jawline: defined, Age: early 20s, Gender: Female, Race: Human, Clothing: Cream-colored robe with subtle floral embroidery along the sleeves and hem, graceful and refined.",
    "سلمى": "Eye Color: amber, Eye Shape: round, Face Shape: round, Nose: button, Hairstyle: short and curly, Hair Color: dark brown, Skin Color: olive, Lips: small, Eyebrows: full, Jawline: soft, Age: mid-20s, Gender: Female, Race: Human, Clothing: Bright yellow dress with geometric blue patterns, casual and cheerful.",
    "هند": "Eye Color: green, Eye Shape: almond, Face Shape: diamond, Nose: long, Hairstyle: long and straight, Hair Color: black, Skin Color: tan, Lips: thin, Eyebrows: thin, Jawline: sharp, Age: late 20s, Gender: Female, Race: Human, Clothing: Rich maroon tunic with intricate gold thread work around the neckline and cuffs, dignified and traditional.",
    "عبلة": "Eye Color: deep brown, Eye Shape: wide, Face Shape: oval, Nose: rounded, Hairstyle: medium length, Hair Color: chestnut, Skin Color: medium, Lips: full, Eyebrows: curved, Jawline: soft, Age: early 30s, Gender: Female, Race: Human, Clothing: Soft blue gown with silver beadwork along the collar and cuffs, elegant and sophisticated."
}

# combine all dictionaries into one
character_descriptions = {**character_descriptions1, **character_descriptions2, **character_descriptions3, **character_descriptions4, **character_descriptions5}

In [46]:
# A list of prompts for DALL-E to generate images of the characters.

dalle_prompts = []
for _, narrative in approved_narrative_texts:
    prompt = narrative  # Start with the base narrative
    descriptions = "\nCharacter descriptions: "

    # Check for each character in the narrative
    for name in character_descriptions:
        if name in narrative:
            # Append each found character's description
            descriptions += f"\n{name}: {character_descriptions[name]}"

    # Only append descriptions if any were found
    if len(descriptions) > len("\nCharacter descriptions: "):
        prompt += descriptions

    dalle_prompts.append(prompt)


In [48]:
# Define a general intro that sets the context for the scenes
general_intro = "Visualizing a scene from مجنون ليلى, set in the classical Arab period, illustrated in the Classical Painting style. \n"


# Generate the detailed prompts with context
dalle_prompts_with_intro = []
for prompt in dalle_prompts:
    # Concatenate the general intro with the existing prompts
    enriched_prompt = general_intro + prompt
    dalle_prompts_with_intro.append(enriched_prompt)

print(len(dalle_prompts_with_intro))


41


In [50]:
# save the prompts to a file
with open('dalle_prompts_with_intro.txt', 'w', encoding='utf-8') as file:
    for prompt in dalle_prompts_with_intro:
        file.write(prompt + '\n')

In [51]:
import os

# Reading the translated narrative texts
file_path = 'approved_narrative_texts_translated.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    translated_narrative_texts = file.readlines()

# Removing any extra newlines
translated_narrative_texts = [line.strip() for line in translated_narrative_texts]

# Generate detailed DALL-E prompts with character descriptions
dalle_prompts = []
for narrative in translated_narrative_texts:
    prompt = narrative  # Start with the base narrative
    descriptions = "\nCharacter descriptions: "

    # Check for each character in the narrative
    for name in character_descriptions:
        if name in narrative:
            # Append each found character's description
            descriptions += f"\n{name}: {character_descriptions[name]}"

    # Only append descriptions if any were found
    if len(descriptions) > len("\nCharacter descriptions: "):
        prompt += descriptions

    dalle_prompts.append(prompt)

# Define a general intro that sets the context for the scenes
general_intro = "Visualizing a scene from مجنون ليلى, set in the classical Arab period, illustrated in the Classical Painting style. \n"

# Generate the detailed prompts with context
dalle_prompts_with_intro_trans = []
for prompt in dalle_prompts:
    # Concatenate the general intro with the existing prompts
    enriched_prompt = general_intro + prompt
    dalle_prompts_with_intro_trans.append(enriched_prompt)


41
Visualizing a scene from مجنون ليلى, set in the classical Arab period, illustrated in the Classical Painting style. 
(A square in front of Al-Mahdi's tents in Bani Amir district - One of the evening gathering places in this square - Boys and girls from the district chat in the early night, girls hold wool and spindles, playing with them as they talk — ليلى comes out of her father's tent when the curtain rises, her hand in ابن ذريح's hand)
Character descriptions: 
ليلى: Eye Color: dark brown, Eye Shape: almond, Face Shape: heart, Nose: small, upturned, Hairstyle: long and wavy, Hair Color: black, Skin Color: fair, Lips: medium, Eyebrows: medium, curved, Jawline: soft, Age: early 20s, Gender: Female, Race: Human, Clothing: Ankle-length soft blue dress with minimal floral embroidery around the neckline and sleeves.
ابن ذريح: Eye Color: amber, Eye Shape: slightly slanted, Face Shape: heart, Nose: refined, Hairstyle: long and flowing, Hair Color: golden brown, Skin Color: fair, Lips: wel

In [53]:
import os
import requests
from io import BytesIO
from PIL import Image
import openai
from datetime import datetime

def generate_image(prompt, image_index):
    """
    Generates a single image based on a given prompt using DALL-E 3 and saves it as a file in the 'images' folder.

    Parameters:
    - prompt (str): The prompt to generate an image for.
    - image_index (int): Index used to generate sequential file names.

    Requires:
    - openai (package): For making API requests to DALL-E.
    - PIL (package): For image processing.
    """

    # Verify OpenAI package version
    required_version = "1.2.3"
    current_version = openai.__version__
    if tuple(map(int, current_version.split("."))) < tuple(map(int, required_version.split("."))):
        raise ValueError(f"OpenAI version {current_version} is less than the required version {required_version}.")

    # Configuration for the image request
    image_params = {
        "model": "dall-e-3",
        "n": 1,
        "size": "1024x1024",
        "prompt": prompt,
    }

    # Initialize OpenAI client using environment variable for API key
    client = openai.OpenAI()

    # Make the request to generate the image
    try:
        images_response = client.images.generate(**image_params)
    except openai.error.OpenAIError as e:
        raise Exception(f"Failed to generate image: {e}")

    # Extract image URL
    image_url = images_response.data[0].url

    # Download and save the image
    response = requests.get(image_url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        filename = f"images1/img_{image_index}.png"
        image.save(filename)
        print(f"Image saved as {filename}")
        print(f"Prompt: {prompt}")
    else:
        raise Exception("Failed to download the image.")

    return filename, prompt


for index, prompt in enumerate(dalle_prompts_with_intro_trans):
    generate_image(prompt, index)

Image saved as images1/img_0.png
Prompt: Visualizing a scene from مجنون ليلى, set in the classical Arab period, illustrated in the Classical Painting style. 
(A square in front of Al-Mahdi's tents in Bani Amir district - One of the evening gathering places in this square - Boys and girls from the district chat in the early night, girls hold wool and spindles, playing with them as they talk — ليلى comes out of her father's tent when the curtain rises, her hand in ابن ذريح's hand)
Character descriptions: 
ليلى: Eye Color: dark brown, Eye Shape: almond, Face Shape: heart, Nose: small, upturned, Hairstyle: long and wavy, Hair Color: black, Skin Color: fair, Lips: medium, Eyebrows: medium, curved, Jawline: soft, Age: early 20s, Gender: Female, Race: Human, Clothing: Ankle-length soft blue dress with minimal floral embroidery around the neckline and sleeves.
ابن ذريح: Eye Color: amber, Eye Shape: slightly slanted, Face Shape: heart, Nose: refined, Hairstyle: long and flowing, Hair Color: gol

In [54]:
# Load the narrative texts from the .npy file
narrative_texts = np.load('uncleaned_narrative_texts.npy', allow_pickle=True)

# Create a list of image file names in order
image_file_names = [f'img_{i}.png' for i in range(len(narrative_texts))]

# Create a list of tuples (narrative_text, image_file_name)
narrative_image_pairs = list(zip(narrative_texts, image_file_names))

# Save the list of tuples as a .npy file
np.save('narrative_image_pairs.npy', narrative_image_pairs)

# Print confirmation message
print("Saved list of tuples as 'narrative_image_pairs.npy'")

Saved list of tuples as 'narrative_image_pairs.npy'
